In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from osgeo import gdal
from osgeo import gdalconst
import os
from osgeo import ogr
from osgeo import osr
import fiona
from ops.ops import load_json, load_opt_image
from tqdm.notebook import tqdm_notebook
from osgeo import gdal_array
from skimage.morphology import disk, dilation, erosion
import numpy as np
from s2cloudless import get_s2_evalscript, S2PixelCloudDetector

In [3]:
#image_folder = 'img/GEE-imgs'

paths = load_json(os.path.join('conf', 'paths.json'))
conf = load_json(os.path.join('conf', 'conf.json'))

opt_path = os.path.join('img', 'opt')

In [4]:
opt_2017 = np.concatenate([
    load_opt_image(os.path.join(opt_path, '2017', 'PA_S2_2017_B1_B2_B3_crop.tif')),
    load_opt_image(os.path.join(opt_path, '2017', 'PA_S2_2017_B4_B5_B6_crop.tif')),
    load_opt_image(os.path.join(opt_path, '2017', 'PA_S2_2017_B7_B8_B8A_crop.tif')),
    load_opt_image(os.path.join(opt_path, '2017', 'PA_S2_2017_B9_B10_B11_crop.tif')),
    load_opt_image(os.path.join(opt_path, '2017', 'PA_S2_2017_B12_crop.tif'))
], axis=-1)

opt_2018 = np.concatenate([
    load_opt_image(os.path.join(opt_path, '2018', 'COPERNICUS_S2_20180721_20180726_B1_B2_B3.tif')),
    load_opt_image(os.path.join(opt_path, '2018', 'COPERNICUS_S2_20180721_20180726_B4_B5_B6.tif')),
    load_opt_image(os.path.join(opt_path, '2018', 'COPERNICUS_S2_20180721_20180726_B7_B8_B8A.tif')),
    load_opt_image(os.path.join(opt_path, '2018', 'COPERNICUS_S2_20180721_20180726_B9_B10_B11.tif')),
    load_opt_image(os.path.join(opt_path, '2018', 'COPERNICUS_S2_20180721_20180726_B12.tif'))
], axis=-1)

opt_2019 = np.concatenate([
    load_opt_image(os.path.join(opt_path, '2019', 'COPERNICUS_S2_20190721_20190726_B1_B2_B3.tif')),
    load_opt_image(os.path.join(opt_path, '2019', 'COPERNICUS_S2_20190721_20190726_B4_B5_B6.tif')),
    load_opt_image(os.path.join(opt_path, '2019', 'COPERNICUS_S2_20190721_20190726_B7_B8_B8A.tif')),
    load_opt_image(os.path.join(opt_path, '2019', 'COPERNICUS_S2_20190721_20190726_B9_B10_B11.tif')),
    load_opt_image(os.path.join(opt_path, '2019', 'COPERNICUS_S2_20190721_20190726_B12.tif'))
], axis=-1)

In [5]:
opt_imgs = [
    opt_2017,
    opt_2018,
    opt_2019
]

In [6]:
image_folder = os.path.join(paths['img'], 'opt')
cmap_folder = os.path.join(paths['img'], 'cmaps')
cloud_detector = S2PixelCloudDetector(threshold=0.4, average_over=4, dilation_size=2, all_bands=True)
for img_i, opt_img in enumerate(opt_imgs):
    opt_data = opt_img/10000
    print(opt_data.shape)
    
    cm_map = cloud_detector.get_cloud_probability_maps(opt_data)
    print(cm_map.shape)
    cm_map = (np.rint(100*cm_map)).astype(np.uint8)


    data = gdal.Open(os.path.join(opt_path, '2017', 'PA_S2_2017_B1_B2_B3_crop.tif'), gdalconst.GA_ReadOnly)
    geo_transform = data.GetGeoTransform()
    x_min = geo_transform[0]
    y_max = geo_transform[3]
    x_max = x_min + geo_transform[1] * data.RasterXSize
    y_min = y_max + geo_transform[5] * data.RasterYSize
    x_res = data.RasterXSize
    y_res = data.RasterYSize

    crs = data.GetSpatialRef()
    proj = data.GetProjection()

    pixel_width = geo_transform[1]

    output_cmap = os.path.join(cmap_folder, f'cmap_{img_i+2017}.tif')

    target_ds_cmap = gdal.GetDriverByName('GTiff').Create(output_cmap, x_res, y_res, 1, gdal.GDT_Byte)

    target_ds_cmap.SetGeoTransform(geo_transform)
    target_ds_cmap.SetSpatialRef(crs)
    target_ds_cmap.SetProjection(proj)
    band_cmap = target_ds_cmap.GetRasterBand(1)
    band_cmap.FlushCache()

    target_ds_cmap.GetRasterBand(1).WriteArray(cm_map)
    target_ds_cmap = None

    print(f'{cm_map.min()} || {cm_map.max()}')

    


(17730, 9203, 13)
(17730, 9203)
0 || 92
(17730, 9203, 13)
(17730, 9203)
0 || 100
(17730, 9203, 13)
(17730, 9203)
0 || 100
